In [18]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from collections import OrderedDict
import pandas as pd
import re
from tqdm import tqdm

from io import StringIO
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [42]:
ts_values = ["replace_first", 
             "replace_question_word", 
             "add_adalah",
             "rule_based",
             "machine_generation_with_rule_based",
             "pure_machine_generation",
             "machine_generation_with_translation"
            ]

for ts_value in ts_values:
    command = (
        f"python inference_qa_with_nli_validation.py "
        f"-d squadid -msc indonli_mnli_squadid-nli -tq entailment_or_neutral -msi 2 -va 3 -th 0.50 -ts {ts_value}"
    )
    print(command)

python inference_qa_with_nli_validation.py -d squadid -msc indonli_mnli_squadid-nli -tq entailment_or_neutral -msi 2 -va 3 -th 0.50 -ts replace_first
python inference_qa_with_nli_validation.py -d squadid -msc indonli_mnli_squadid-nli -tq entailment_or_neutral -msi 2 -va 3 -th 0.50 -ts replace_question_word
python inference_qa_with_nli_validation.py -d squadid -msc indonli_mnli_squadid-nli -tq entailment_or_neutral -msi 2 -va 3 -th 0.50 -ts add_adalah
python inference_qa_with_nli_validation.py -d squadid -msc indonli_mnli_squadid-nli -tq entailment_or_neutral -msi 2 -va 3 -th 0.50 -ts rule_based
python inference_qa_with_nli_validation.py -d squadid -msc indonli_mnli_squadid-nli -tq entailment_or_neutral -msi 2 -va 3 -th 0.50 -ts machine_generation_with_rule_based
python inference_qa_with_nli_validation.py -d squadid -msc indonli_mnli_squadid-nli -tq entailment_or_neutral -msi 2 -va 3 -th 0.50 -ts pure_machine_generation
python inference_qa_with_nli_validation.py -d squadid -msc indonli_

# prepare

In [2]:
prefix = "https://huggingface.co/muhammadravi251001/fine-tuned-FilteringNLI-"
suffix = "/blob/main/results/evaluation/metric_result.txt"

suffix_df = "/raw/main/results/output/output_df.csv"
suffix_squadid_df = "/resolve/main/results/output/output_df.csv"

In [20]:
def em_and_f1(p_url, acc, id="L1", rounded_to_num=5):

    page = requests.get(p_url)
    soup = BeautifulSoup(page.content)

    tr = soup.find('tr', attrs={'id': id})
    td = tr.find(attrs={'class': 'overflow-visible whitespace-pre px-3'}).text
    x = re.findall("\d+\.\d+", td)

    if acc == True: return round(float(x[0]), rounded_to_num)
    else: return round(float(x[1]), rounded_to_num)

In [81]:
def take_all_url_nli_validator(data, df=False):
    
    if not df:
        baseline = f"{prefix}indonli_mnli-{data}-TQ2-TS4-MS3-VA0-TH0.0{suffix}"
    else:
        if data == "idkmrc" or data == "tydiqaid":
            baseline = f"{prefix}indonli_mnli-{data}-TQ2-TS4-MS3-VA0-TH0.0{suffix_df}"
        elif data == "squadid":
            baseline = f"{prefix}indonli_mnli-{data}-TQ2-TS4-MS3-VA0-TH0.0{suffix_squadid_df}"
    
    msc_arr = [f"indonli_mnli_{data}-nli"]
    ts_arr = [1, 2, 3, 4, 5, 6, 7, 8]
    
    if data == "idkmrc":
        tq_arr = [1]
        msi_arr = [1]
        var_arr = [2]
        th_arr = [0.75]
    
    elif data == "tydiqaid":
        tq_arr = [1]
        msi_arr = [2]
        var_arr = [3]
        th_arr = [0.5]
        
    elif data == "squadid":
        tq_arr = [2]
        msi_arr = [2]
        var_arr = [3]
        th_arr = [0.5]

    url_arr = []
    url_arr.append(baseline)

    for msc in msc_arr:
        for tq in tq_arr:
            for ts in ts_arr:
                for msi in msi_arr:
                    for var in var_arr:
                        for th in th_arr:

                            if msc == "indonli" or msc == "indonli_mnli":
                                var = 1
                                th = 0.0

                            if var == 1:
                                th = 0.0

                            if (var == 3) and (msi == 1):
                                continue

                            if not df:
                                url_arr.append(f"{prefix}{msc}-{data}-TQ{tq}-TS{ts}-MS{msi}-VA{var}-TH{th}{suffix}")
                            
                            else:
                                
                                if data == "idkmrc" or data == "tydiqaid":
                                    url_arr.append(f"{prefix}{msc}-{data}-TQ{tq}-TS{ts}-MS{msi}-VA{var}-TH{th}{suffix_df}")
                                
                                elif data == "squadid":
                                    url_arr.append(f"{prefix}{msc}-{data}-TQ{tq}-TS{ts}-MS{msi}-VA{var}-TH{th}{suffix_squadid_df}")
    
    url_arr = list(OrderedDict.fromkeys(url_arr))
    print("Amount of url:", len(url_arr))
    return url_arr

In [61]:
def take_dataframe(url): 
    
    response = requests.get(url)
    
    if response.status_code == 200:
        csv_data = StringIO(response.text)
        df = pd.read_csv(csv_data, index_col=0)
        df = df[['Prediction Answer', 'Rec. Pred Answer', 'Gold Answer', 'Properties']]
        df.fillna('', inplace=True)
    
    else:
        print("Failed to download CSV")
    
    return df

In [62]:
def take_accepted_and_rejected_qa_with_nli(url):
    
    df = take_dataframe(url)
    
    right_pred_model_qa_and_accepted_by_nli_ctr = 0
    right_pred_model_qa_and_rejected_by_nli_ctr = 0
    wrong_pred_model_qa_and_accepted_by_nli_ctr = 0
    wrong_pred_model_qa_and_rejected_by_nli_ctr = 0
    
    for i in range(len(df)):
        
        gold_answer = df['Gold Answer'][i]
        qa_generated_answer = eval(df['Rec. Pred Answer'][i])[0]
        nli_validated_answer = df['Prediction Answer'][i]
    
        # Jawaban dari model qa yg benar & di-accept sama model NLI
        right_pred_model_qa_and_accepted_by_nli = (gold_answer == qa_generated_answer) and \
            (qa_generated_answer == nli_validated_answer)

        # Jawaban dari model qa yg benar & di-reject sama model NLI
        right_pred_model_qa_and_rejected_by_nli = (gold_answer == qa_generated_answer) and \
            (qa_generated_answer != nli_validated_answer)

        # Jawaban dari model qa yg salah & di-accept sama model NLI
        wrong_pred_model_qa_and_accepted_by_nli = (gold_answer != qa_generated_answer) and \
            (qa_generated_answer == nli_validated_answer)

        # Jawaban dari model qa yg salah & di-reject sama model NLI
        wrong_pred_model_qa_and_rejected_by_nli = (gold_answer != qa_generated_answer) and \
            (qa_generated_answer != nli_validated_answer)
        
        if right_pred_model_qa_and_accepted_by_nli:
            right_pred_model_qa_and_accepted_by_nli_ctr += 1
        elif right_pred_model_qa_and_rejected_by_nli:
            right_pred_model_qa_and_rejected_by_nli_ctr += 1
        elif wrong_pred_model_qa_and_accepted_by_nli:
            wrong_pred_model_qa_and_accepted_by_nli_ctr += 1
        elif wrong_pred_model_qa_and_rejected_by_nli:
            wrong_pred_model_qa_and_rejected_by_nli_ctr += 1
    
    return right_pred_model_qa_and_accepted_by_nli_ctr, right_pred_model_qa_and_rejected_by_nli_ctr, \
            wrong_pred_model_qa_and_accepted_by_nli_ctr, wrong_pred_model_qa_and_rejected_by_nli_ctr

In [63]:
def extract_values_nli_validator(url, data):

    msc_list = f"indonli_mnli_{data}-nli|indonli_mnli|indonli"
    pattern = re.compile(r'.*FilteringNLI-({})-{}-TQ(\d+)-TS(\d+)-MS([\d]+)-VA([\d]+)-TH([\d.]+).*'.format(msc_list, data))
    match = pattern.match(url)
    
    if match:
        msc = match.group(1)
        TQ = int(match.group(2))
        TS = int(match.group(3))
        MSI = int(match.group(4))
        VA = int(match.group(5))
        
        TH = float(match.group(6))
        if TH == '0.00':
            TH = '0.0'
        elif TH == '0.50':
            TH = '0.5'

        EM = em_and_f1(url, True)
        F1 = em_and_f1(url, False)

        return msc, data, TQ, TS, MSI, VA, TH, EM, F1
    
    else:
        return None

In [64]:
def create_table_nli_validator(url_list, url_list_df, data):
    
    data_list = []
    for url in tqdm(url_list, desc="Processing first URLs", unit="URL"):
        extracted_values = extract_values_nli_validator(url, data)
        data_list.append(extracted_values)

    df = pd.DataFrame(data_list, columns=['MSC', 'data', 'TQ', 'TS', 'MSI', 'VA', 'TH', 'EM', 'F1'])
    
    for idx, url_df in enumerate(tqdm(url_list_df, desc="Processing second URLs", unit="URL")):
        extracted_values = take_accepted_and_rejected_qa_with_nli(url_df)
        df.loc[idx, ['RA', 'RR', 'WA', 'WR']] = extracted_values
    
    df[['RA', 'RR', 'WA', 'WR']] = df[['RA', 'RR', 'WA', 'WR']].astype(int)
    
    df.loc[0, 'MSC'] = "Baseline"
    df.loc[0, 'TQ'] = "BL"
    df.loc[0, 'TS'] = "BL"
    df.loc[0, 'MSI'] = "BL"
    df.loc[0, 'VA'] = "BL"
    df.loc[0, 'TH'] = "BL"
    
    return df

# idkmrc

In [65]:
# 1,4,1,2,0.75 (V)

In [66]:
data = "idkmrc"
url_list = take_all_url_nli_validator(data)
url_list_df = take_all_url_nli_validator(data, df=True)
table = pd.DataFrame(create_table_nli_validator(url_list, url_list_df, data))
#table.to_excel(f'{data}-nli-validator.xlsx', index=False)
table

Amount of url: 9
Amount of url: 9


Processing second URLs: 100%|████████████████████████████████████████████████████████████| 9/9 [00:05<00:00,  1.54URL/s]


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1,RA,RR,WA,WR
0,Baseline,idkmrc,BL,BL,BL,BL,BL,75.82938,84.25128,640,0,204,0
1,indonli_mnli_idkmrc-nli,idkmrc,1,1,1,2,0.75,71.44550,78.45857,586,54,152,52
2,indonli_mnli_idkmrc-nli,idkmrc,1,2,1,2,0.75,71.68246,78.79067,590,50,155,49
3,indonli_mnli_idkmrc-nli,idkmrc,1,3,1,2,0.75,73.81517,81.11026,611,29,172,32
4,indonli_mnli_idkmrc-nli,idkmrc,1,4,1,2,0.75,75.94787,84.16834,639,1,198,6
5,indonli_mnli_idkmrc-nli,idkmrc,1,5,1,2,0.75,74.76303,82.35951,620,20,179,25
6,indonli_mnli_idkmrc-nli,idkmrc,1,6,1,2,0.75,65.63981,70.14204,540,100,130,74
7,indonli_mnli_idkmrc-nli,idkmrc,1,7,1,2,0.75,67.06161,72.80594,551,89,136,68
8,indonli_mnli_idkmrc-nli,idkmrc,1,8,1,2,0.75,55.56872,58.81457,449,191,100,104


In [67]:
print(f"{data} sorted by EM")
sorted_table = table.sort_values(by='EM', ascending=False)
sorted_table

idkmrc sorted by EM


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1,RA,RR,WA,WR
4,indonli_mnli_idkmrc-nli,idkmrc,1,4,1,2,0.75,75.94787,84.16834,639,1,198,6
0,Baseline,idkmrc,BL,BL,BL,BL,BL,75.82938,84.25128,640,0,204,0
5,indonli_mnli_idkmrc-nli,idkmrc,1,5,1,2,0.75,74.76303,82.35951,620,20,179,25
3,indonli_mnli_idkmrc-nli,idkmrc,1,3,1,2,0.75,73.81517,81.11026,611,29,172,32
2,indonli_mnli_idkmrc-nli,idkmrc,1,2,1,2,0.75,71.68246,78.79067,590,50,155,49
1,indonli_mnli_idkmrc-nli,idkmrc,1,1,1,2,0.75,71.44550,78.45857,586,54,152,52
7,indonli_mnli_idkmrc-nli,idkmrc,1,7,1,2,0.75,67.06161,72.80594,551,89,136,68
6,indonli_mnli_idkmrc-nli,idkmrc,1,6,1,2,0.75,65.63981,70.14204,540,100,130,74
8,indonli_mnli_idkmrc-nli,idkmrc,1,8,1,2,0.75,55.56872,58.81457,449,191,100,104


In [68]:
print(f"{data} sorted by F1")
sorted_table = table.sort_values(by='F1', ascending=False)
sorted_table

idkmrc sorted by F1


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1,RA,RR,WA,WR
0,Baseline,idkmrc,BL,BL,BL,BL,BL,75.82938,84.25128,640,0,204,0
4,indonli_mnli_idkmrc-nli,idkmrc,1,4,1,2,0.75,75.94787,84.16834,639,1,198,6
5,indonli_mnli_idkmrc-nli,idkmrc,1,5,1,2,0.75,74.76303,82.35951,620,20,179,25
3,indonli_mnli_idkmrc-nli,idkmrc,1,3,1,2,0.75,73.81517,81.11026,611,29,172,32
2,indonli_mnli_idkmrc-nli,idkmrc,1,2,1,2,0.75,71.68246,78.79067,590,50,155,49
1,indonli_mnli_idkmrc-nli,idkmrc,1,1,1,2,0.75,71.44550,78.45857,586,54,152,52
7,indonli_mnli_idkmrc-nli,idkmrc,1,7,1,2,0.75,67.06161,72.80594,551,89,136,68
6,indonli_mnli_idkmrc-nli,idkmrc,1,6,1,2,0.75,65.63981,70.14204,540,100,130,74
8,indonli_mnli_idkmrc-nli,idkmrc,1,8,1,2,0.75,55.56872,58.81457,449,191,100,104


In [69]:
# idkmrc hasil nya kurang bagus, lebih baik tetap di just_concat_answer_and_question

# tydiqaid

In [70]:
# 1,4,2,3,0.50 (V)

In [72]:
data = "tydiqaid"
url_list = take_all_url_nli_validator(data)
url_list_df = take_all_url_nli_validator(data, df=True)
table = pd.DataFrame(create_table_nli_validator(url_list, url_list_df, data))
#table.to_excel(f'{data}-nli-validator.xlsx', index=False)
table

Amount of url: 9
Amount of url: 9


Processing second URLs: 100%|████████████████████████████████████████████████████████████| 9/9 [00:07<00:00,  1.24URL/s]


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1,RA,RR,WA,WR
0,Baseline,tydiqaid,BL,BL,BL,BL,BL,61.28655,76.47617,524,0,331,0
1,indonli_mnli_tydiqaid-nli,tydiqaid,1,1,2,3,0.5,57.30994,73.35205,483,41,276,55
2,indonli_mnli_tydiqaid-nli,tydiqaid,1,2,2,3,0.5,57.30994,73.27496,483,41,275,56
3,indonli_mnli_tydiqaid-nli,tydiqaid,1,3,2,3,0.5,59.64912,75.93470,501,23,288,43
4,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,2,3,0.5,61.28655,77.45290,523,1,305,26
5,indonli_mnli_tydiqaid-nli,tydiqaid,1,5,2,3,0.5,60.00000,75.99800,510,14,297,34
6,indonli_mnli_tydiqaid-nli,tydiqaid,1,6,2,3,0.5,53.09942,70.24392,448,76,263,68
7,indonli_mnli_tydiqaid-nli,tydiqaid,1,7,2,3,0.5,51.22807,67.93912,426,98,251,80
8,indonli_mnli_tydiqaid-nli,tydiqaid,1,8,2,3,0.5,46.43275,64.44547,387,137,209,122


In [73]:
print(f"{data} sorted by EM")
sorted_table = table.sort_values(by='EM', ascending=False)
sorted_table

tydiqaid sorted by EM


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1,RA,RR,WA,WR
0,Baseline,tydiqaid,BL,BL,BL,BL,BL,61.28655,76.47617,524,0,331,0
4,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,2,3,0.5,61.28655,77.45290,523,1,305,26
5,indonli_mnli_tydiqaid-nli,tydiqaid,1,5,2,3,0.5,60.00000,75.99800,510,14,297,34
3,indonli_mnli_tydiqaid-nli,tydiqaid,1,3,2,3,0.5,59.64912,75.93470,501,23,288,43
1,indonli_mnli_tydiqaid-nli,tydiqaid,1,1,2,3,0.5,57.30994,73.35205,483,41,276,55
2,indonli_mnli_tydiqaid-nli,tydiqaid,1,2,2,3,0.5,57.30994,73.27496,483,41,275,56
6,indonli_mnli_tydiqaid-nli,tydiqaid,1,6,2,3,0.5,53.09942,70.24392,448,76,263,68
7,indonli_mnli_tydiqaid-nli,tydiqaid,1,7,2,3,0.5,51.22807,67.93912,426,98,251,80
8,indonli_mnli_tydiqaid-nli,tydiqaid,1,8,2,3,0.5,46.43275,64.44547,387,137,209,122


In [74]:
print(f"{data} sorted by F1")
sorted_table = table.sort_values(by='F1', ascending=False)
sorted_table

tydiqaid sorted by F1


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1,RA,RR,WA,WR
4,indonli_mnli_tydiqaid-nli,tydiqaid,1,4,2,3,0.5,61.28655,77.45290,523,1,305,26
0,Baseline,tydiqaid,BL,BL,BL,BL,BL,61.28655,76.47617,524,0,331,0
5,indonli_mnli_tydiqaid-nli,tydiqaid,1,5,2,3,0.5,60.00000,75.99800,510,14,297,34
3,indonli_mnli_tydiqaid-nli,tydiqaid,1,3,2,3,0.5,59.64912,75.93470,501,23,288,43
1,indonli_mnli_tydiqaid-nli,tydiqaid,1,1,2,3,0.5,57.30994,73.35205,483,41,276,55
2,indonli_mnli_tydiqaid-nli,tydiqaid,1,2,2,3,0.5,57.30994,73.27496,483,41,275,56
6,indonli_mnli_tydiqaid-nli,tydiqaid,1,6,2,3,0.5,53.09942,70.24392,448,76,263,68
7,indonli_mnli_tydiqaid-nli,tydiqaid,1,7,2,3,0.5,51.22807,67.93912,426,98,251,80
8,indonli_mnli_tydiqaid-nli,tydiqaid,1,8,2,3,0.5,46.43275,64.44547,387,137,209,122


In [ ]:
# tydiqaid hasil nya kurang bagus, lebih baik tetap di just_concat_answer_and_question

# squadid

In [75]:
# 2,4,2,3,0.50 (V)

In [82]:
data = "squadid"
url_list = take_all_url_nli_validator(data)
url_list_df = take_all_url_nli_validator(data, df=True)
table = pd.DataFrame(create_table_nli_validator(url_list, url_list_df, data))
#table.to_excel(f'{data}-nli-validator.xlsx', index=False)
table

Amount of url: 9
Amount of url: 9


Processing second URLs: 100%|████████████████████████████████████████████████████████████| 9/9 [02:47<00:00, 18.66s/URL]


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1,RA,RR,WA,WR
0,Baseline,squadid,BL,BL,BL,BL,BL,47.91544,67.07638,5689,0,6184,0
1,indonli_mnli_squadid-nli,squadid,2,1,2,3,0.5,40.95848,60.09626,4771,918,4906,1278
2,indonli_mnli_squadid-nli,squadid,2,2,2,3,0.5,42.60086,61.65735,4985,704,5132,1052
3,indonli_mnli_squadid-nli,squadid,2,3,2,3,0.5,46.86263,66.44941,5515,174,5558,626
4,indonli_mnli_squadid-nli,squadid,2,4,2,3,0.5,47.80595,67.05412,5661,28,6032,152
5,indonli_mnli_squadid-nli,squadid,2,5,2,3,0.5,47.66276,67.27186,5627,62,5723,461
6,indonli_mnli_squadid-nli,squadid,2,6,2,3,0.5,42.83669,62.00118,5031,658,5231,953
7,indonli_mnli_squadid-nli,squadid,2,7,2,3,0.5,45.98669,65.09704,5429,260,5785,399
8,indonli_mnli_squadid-nli,squadid,2,8,2,3,0.5,40.22572,58.61736,4712,977,4983,1201


In [83]:
print(f"{data} sorted by EM")
sorted_table = table.sort_values(by='EM', ascending=False)
sorted_table

squadid sorted by EM


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1,RA,RR,WA,WR
0,Baseline,squadid,BL,BL,BL,BL,BL,47.91544,67.07638,5689,0,6184,0
4,indonli_mnli_squadid-nli,squadid,2,4,2,3,0.5,47.80595,67.05412,5661,28,6032,152
5,indonli_mnli_squadid-nli,squadid,2,5,2,3,0.5,47.66276,67.27186,5627,62,5723,461
3,indonli_mnli_squadid-nli,squadid,2,3,2,3,0.5,46.86263,66.44941,5515,174,5558,626
7,indonli_mnli_squadid-nli,squadid,2,7,2,3,0.5,45.98669,65.09704,5429,260,5785,399
6,indonli_mnli_squadid-nli,squadid,2,6,2,3,0.5,42.83669,62.00118,5031,658,5231,953
2,indonli_mnli_squadid-nli,squadid,2,2,2,3,0.5,42.60086,61.65735,4985,704,5132,1052
1,indonli_mnli_squadid-nli,squadid,2,1,2,3,0.5,40.95848,60.09626,4771,918,4906,1278
8,indonli_mnli_squadid-nli,squadid,2,8,2,3,0.5,40.22572,58.61736,4712,977,4983,1201


In [84]:
print(f"{data} sorted by F1")
sorted_table = table.sort_values(by='F1', ascending=False)
sorted_table

squadid sorted by F1


,MSC,data,TQ,TS,MSI,VA,TH,EM,F1,RA,RR,WA,WR
5,indonli_mnli_squadid-nli,squadid,2,5,2,3,0.5,47.66276,67.27186,5627,62,5723,461
0,Baseline,squadid,BL,BL,BL,BL,BL,47.91544,67.07638,5689,0,6184,0
4,indonli_mnli_squadid-nli,squadid,2,4,2,3,0.5,47.80595,67.05412,5661,28,6032,152
3,indonli_mnli_squadid-nli,squadid,2,3,2,3,0.5,46.86263,66.44941,5515,174,5558,626
7,indonli_mnli_squadid-nli,squadid,2,7,2,3,0.5,45.98669,65.09704,5429,260,5785,399
6,indonli_mnli_squadid-nli,squadid,2,6,2,3,0.5,42.83669,62.00118,5031,658,5231,953
2,indonli_mnli_squadid-nli,squadid,2,2,2,3,0.5,42.60086,61.65735,4985,704,5132,1052
1,indonli_mnli_squadid-nli,squadid,2,1,2,3,0.5,40.95848,60.09626,4771,918,4906,1278
8,indonli_mnli_squadid-nli,squadid,2,8,2,3,0.5,40.22572,58.61736,4712,977,4983,1201
